In [61]:
import pandas as pd
import numpy as np
import itertools

In [62]:
file_list = ['carSales1', 'carSales2', 'Census', 'Emission', 'NBA', 'COVID-19']
time_list = ['Year', 'Year', 'Birthday', 'Year', 'year', 'date']
time_formatting = [True, False, True, False, False, False]
measure_list = ['Sales', 'Price', 'Count of persons', 'CO2 (kt)']
feature_list = [['Year', 'Brand', 'Category'],
                ['Year', 'Brand', 'Body', 'Engine Type', 'Model', 'Registration'],
                ['Birthday', 'Age Segment', 'Marital Status', 'Sex', 'Age Group'],
                ['Year', 'State', 'Producer Type', 'Energy Source'],
                ['name','year','team_name','age','lg_name','pos_name'],
                ['date','county','state']]
file_index = 3
root = 'C:\\Users\\52497\\Dropbox\\VIS\\Insight\\'
raw_data = root + "Data\\" + file_list[file_index] + ".csv"
res_data = root + "res\\insights_" + file_list[file_index] + ".csv"

In [63]:
# Record file processing
record = pd.read_csv(raw_data)
# record[time_list[file_index]] = pd.to_datetime(record[time_list[file_index]]).dt.date
if time_formatting[file_index]:
    record[time_list[file_index]] = record[time_list[file_index]].apply(lambda year_string: year_string.split('/')[0])

In [64]:
record['cid'] = record.index
record

,Year,State,Producer Type,Energy Source,CO2 (kt),SO2 kt,NOx kt,cid
0,1990,AK,Commercial Cogen,Coal,821.929,13.191,3.009,0
1,1991,AK,Commercial Cogen,Coal,848.745,8.359,3.146,1
2,1992,AK,Commercial Cogen,Coal,860.878,8.469,3.195,2
3,1993,AK,Commercial Cogen,Coal,858.244,8.393,3.208,3
4,1994,AK,Commercial Cogen,Coal,866.661,8.880,3.238,4
...,...,...,...,...,...,...,...,...
41151,2014,WY,Utility Sector Non-Cogen,Natural Gas,27.760,0.000,0.056,41151
41152,2015,WY,Utility Sector Non-Cogen,Natural Gas,0.395,0.000,0.000,41152
41153,2005,WY,Utility Sector Non-Cogen,Petroleum,0.000,0.002,0.023,41153
41154,2006,WY,Utility Sector Non-Cogen,Petroleum,0.317,0.002,0.027,41154


In [65]:
subspace_df = record[feature_list[file_index] + ['cid']].drop_duplicates()
subspace_df

,Year,State,Producer Type,Energy Source,cid
0,1990,AK,Commercial Cogen,Coal,0
1,1991,AK,Commercial Cogen,Coal,1
2,1992,AK,Commercial Cogen,Coal,2
3,1993,AK,Commercial Cogen,Coal,3
4,1994,AK,Commercial Cogen,Coal,4
...,...,...,...,...,...
41151,2014,WY,Utility Sector Non-Cogen,Natural Gas,41151
41152,2015,WY,Utility Sector Non-Cogen,Natural Gas,41152
41153,2005,WY,Utility Sector Non-Cogen,Petroleum,41153
41154,2006,WY,Utility Sector Non-Cogen,Petroleum,41154


In [66]:
for feature in feature_list[file_index]:
    temp_df = subspace_df.copy()
    temp_df[feature] = "*"
    temp_df['cid'] = None
    subspace_df = pd.concat([temp_df, subspace_df], ignore_index=True).drop_duplicates()
subspace_df

,Year,State,Producer Type,Energy Source,cid
0,*,*,*,*,None
8,1990,*,*,*,None
9,1991,*,*,*,None
10,1992,*,*,*,None
11,1993,*,*,*,None
...,...,...,...,...,...
102559,2014,WY,Utility Sector Non-Cogen,Natural Gas,41151
102560,2015,WY,Utility Sector Non-Cogen,Natural Gas,41152
102561,2005,WY,Utility Sector Non-Cogen,Petroleum,41153
102562,2006,WY,Utility Sector Non-Cogen,Petroleum,41154


In [67]:
subspace_df = subspace_df.reset_index(drop=True)
subspace_df

,Year,State,Producer Type,Energy Source,cid
0,*,*,*,*,None
1,1990,*,*,*,None
2,1991,*,*,*,None
3,1992,*,*,*,None
4,1993,*,*,*,None
...,...,...,...,...,...
59866,2014,WY,Utility Sector Non-Cogen,Natural Gas,41151
59867,2015,WY,Utility Sector Non-Cogen,Natural Gas,41152
59868,2005,WY,Utility Sector Non-Cogen,Petroleum,41153
59869,2006,WY,Utility Sector Non-Cogen,Petroleum,41154


In [68]:
from tqdm import tqdm
subspace_record_list = []
for sub_index, sub_line in tqdm(subspace_df.iterrows(), total=subspace_df.shape[0]):
    if sub_line['cid'] is not None :
        subspace_record_list.append([sub_index, sub_line['cid']])
    else:
        filter = {i: sub_line[i] for i in feature_list[file_index] if sub_line[i] != '*'}
        qry = ' and '.join(["`{}` == '{}'".format(k,v) for k,v in filter.items()])
        if qry == '':
            for i in record['cid'].tolist():
                subspace_record_list.append([sub_index, i])
        else:
            for i in record.query(qry)['cid'].tolist():
                subspace_record_list.append([sub_index, i])
subspace_record_df = pd.DataFrame(subspace_record_list, columns=['sid', 'cid'])

100%|██████████| 59871/59871 [00:58<00:00, 1016.71it/s] 


In [69]:
subspace_record_df

,sid,cid
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
658491,59866,41151
658492,59867,41152
658493,59868,41153
658494,59869,41154


In [70]:
subspace_df['sid'] = subspace_df.index
subspace_df.drop(['cid'], axis=1, inplace=True)
subspace_df

,Year,State,Producer Type,Energy Source,sid
0,*,*,*,*,0
1,1990,*,*,*,1
2,1991,*,*,*,2
3,1992,*,*,*,3
4,1993,*,*,*,4
...,...,...,...,...,...
59866,2014,WY,Utility Sector Non-Cogen,Natural Gas,59866
59867,2015,WY,Utility Sector Non-Cogen,Natural Gas,59867
59868,2005,WY,Utility Sector Non-Cogen,Petroleum,59868
59869,2006,WY,Utility Sector Non-Cogen,Petroleum,59869


In [71]:
insight_df = pd.read_csv(res_data)
insight_df['iid'] = insight_df.index
insight_df

,Producer Type,Energy Source,Year,State,breakdown,breakdown_value,measure,sig,impact,insight,insight_type,score,iid
0,*,*,*,MS,Year,-1,CO2 (kt),0.561615,0.014724,trend,shape,0.008269,0
1,Industrial Cogen,Other,*,*,Year,-1,CO2 (kt),0.300003,0.029060,trend,shape,0.008718,1
2,*,*,*,IA,Year,-1,CO2 (kt),0.494322,0.020945,trend,shape,0.010353,2
3,*,*,*,UT,Year,-1,CO2 (kt),0.565674,0.015551,trend,shape,0.008797,3
4,*,*,*,CO,Year,-1,CO2 (kt),0.521803,0.017251,trend,shape,0.009002,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
694,Electric Utility,Coal,*,*,Year,-1,CO2 (kt);SO2 kt,0.650993,0.051414,cross measure correlation,compound,0.033470,694
695,Electric Utility,*,*,*,State,-1,CO2 (kt);SO2 kt,0.680802,0.211148,cross measure correlation,compound,0.143750,695
696,Industrial Non-Cogen,*,*,*,Year,-1,CO2 (kt);SO2 kt,0.560324,0.057732,cross measure correlation,compound,0.032348,696
697,*,*,2014,*,State,-1,CO2 (kt);SO2 kt,0.724247,0.043590,cross measure correlation,compound,0.031570,697


In [72]:
insight_df = insight_df.merge(subspace_df, how='inner', on=feature_list[file_index])
insight_df

,Producer Type,Energy Source,Year,State,breakdown,breakdown_value,measure,sig,impact,insight,insight_type,score,iid,sid
0,*,*,*,MS,Year,-1,CO2 (kt),0.561615,0.014724,trend,shape,0.008269,0,48
1,Industrial Cogen,Other,*,*,Year,-1,CO2 (kt),0.300003,0.029060,trend,shape,0.008718,1,16212
2,Industrial Cogen,Other,*,*,Year,2003,CO2 (kt),1.000000,0.029060,change point,shape,0.029060,122,16212
3,Industrial Cogen,Other,*,*,Year,2006,CO2 (kt),0.998364,0.029060,outlier,shape,0.029013,174,16212
4,Industrial Cogen,Other,*,*,Year,-1,SO2 kt,0.454138,0.029060,trend,shape,0.013197,305,16212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,Industrial Non-Cogen,Natural Gas,*,*,Year,2013,CO2 (kt),0.999455,0.021042,outlier,shape,0.021030,153,16218
574,*,*,*,NH,Year,-1,SO2 kt,0.668848,0.017349,trend,shape,0.011604,301,51
575,*,*,*,CT,Year,-1,SO2 kt,0.682129,0.019681,trend,shape,0.013425,304,32
576,*,*,*,TN,Year,-1,SO2 kt,0.952682,0.018175,trend,shape,0.017315,320,62


In [73]:
insight_df.drop(feature_list[file_index], inplace=True, axis=1)
insight_df

,breakdown,breakdown_value,measure,sig,impact,insight,insight_type,score,iid,sid
0,Year,-1,CO2 (kt),0.561615,0.014724,trend,shape,0.008269,0,48
1,Year,-1,CO2 (kt),0.300003,0.029060,trend,shape,0.008718,1,16212
2,Year,2003,CO2 (kt),1.000000,0.029060,change point,shape,0.029060,122,16212
3,Year,2006,CO2 (kt),0.998364,0.029060,outlier,shape,0.029013,174,16212
4,Year,-1,SO2 kt,0.454138,0.029060,trend,shape,0.013197,305,16212
...,...,...,...,...,...,...,...,...,...,...
573,Year,2013,CO2 (kt),0.999455,0.021042,outlier,shape,0.021030,153,16218
574,Year,-1,SO2 kt,0.668848,0.017349,trend,shape,0.011604,301,51
575,Year,-1,SO2 kt,0.682129,0.019681,trend,shape,0.013425,304,32
576,Year,-1,SO2 kt,0.952682,0.018175,trend,shape,0.017315,320,62


In [74]:
record_path = root + "backend_data\\record_" + file_list[file_index] + ".csv"
insight_path = root + "backend_data\\insight_" + file_list[file_index] + ".csv"
subspace_path = root + "backend_data\\subspace_" + file_list[file_index] + ".csv"
sid_cid_path = root + "backend_data\\sid_cid_" + file_list[file_index] + ".csv"
with open(record_path, 'w+') as file:
    record.to_csv(file, index=False, line_terminator='\n')
with open(insight_path, 'w+') as file:
    insight_df.to_csv(file, index=False, line_terminator='\n')
with open(subspace_path, 'w+') as file:
    subspace_df.to_csv(file, index=False, line_terminator='\n')
with open(sid_cid_path, 'w+') as file:
    subspace_record_df.to_csv(file, index=False, line_terminator='\n')

In [75]:
# file_index = 2
# record_path = root + "backend_data\\record_" + file_list[file_index] + ".csv"
# insight_path = root + "backend_data\\insight_" + file_list[file_index] + ".csv"
# subspace_path = root + "backend_data\\subspace_" + file_list[file_index] + ".csv"
# sid_cid_path = root + "backend_data\\sid_cid_" + file_list[file_index] + ".csv"
# iid_sid_path = root + "backend_data\\iid_sid_" + file_list[file_index] + ".csv"
#
# insight_df = pd.read_csv(insight_path)
# iid_sid_df = pd.read_csv(iid_sid_path)
# insight_df = insight_df.merge(iid_sid_df, on='iid', how='inner')
# with open(insight_path, 'w+') as file:
#     insight_df.to_csv(file, index=False, line_terminator='\n')
